In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime

tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
!rm -rf ./logs/GRU #dont run this line unless you want to delete the tensorboard logs

In [3]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [4]:
with open("../archive/ProcessData.npy", 'rb') as f:
        training_data_x = np.load(f)
        test_x = np.load(f)
        training_data_y = np.load(f)
        test_y = np.load(f)
        
training_data_x = convert_3D(training_data_x) 
test_x = convert_3D(test_x)

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

In [5]:
LSTMmodel = Sequential([
    LSTM(64, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    #Dropout(0.3),
    LSTM(64),
    #Dropout(0.3),
    Dense(1, activation='sigmoid')
])

LSTMmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.000003),
    metrics=["accuracy", tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalseNegatives(),
             tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives()]
)

log_dir = "logs/LSTM/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

csv_logger = CSVLogger('keras-models/KerasLSTM/training.csv',separator=",", append=False)

LSTMmodel.fit(training_data_x, training_data_y, epochs=10, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = LSTMmodel
model.save('keras-models/KerasLSTM')

Epoch 1/10
221/221 [==============================] - 52s 216ms/step - loss: 0.6936 - accuracy: 0.4926 - auc: 0.5503 - precision: 0.4385 - recall: 0.0699 - false_negatives: 3272.0000 - false_positives: 315.0000 - true_negatives: 3237.0000 - true_positives: 246.0000 - val_loss: 0.6923 - val_accuracy: 0.4957 - val_auc: 0.5658 - val_precision: 0.5093 - val_recall: 0.0718 - val_false_negatives: 1422.0000 - val_false_positives: 106.0000 - val_true_negatives: 1392.0000 - val_true_positives: 110.0000
Epoch 2/10
221/221 [==============================] - 44s 199ms/step - loss: 0.6912 - accuracy: 0.5020 - auc: 0.5875 - precision: 0.4970 - recall: 0.0713 - false_negatives: 3267.0000 - false_positives: 254.0000 - true_negatives: 3298.0000 - true_positives: 251.0000 - val_loss: 0.6901 - val_accuracy: 0.5036 - val_auc: 0.6417 - val_precision: 0.5722 - val_recall: 0.0725 - val_false_negatives: 1421.0000 - val_false_positives: 83.0000 - val_true_negatives: 1415.0000 - val_true_positives: 111.0000
Epo

INFO:tensorflow:Assets written to: keras-models/KerasLSTM/assets


INFO:tensorflow:Assets written to: keras-models/KerasLSTM/assets


In [6]:
%tensorboard --logdir logs/LSTM/fit